In [1]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path = '/Users/xuchengwei/Desktop/汇丰论文/空气质量/IHME_GBD_2021_AIR_POLLUTION_1990_2021_NO2'

In [3]:
# 打开TIFF文件
with rasterio.open("/Users/xuchengwei/Desktop/汇丰论文/空气质量/IHME_GBD_2021_AIR_POLLUTION_1990_2021_NO2/IHME_GBD_2021_AIR_POLLUTION_1990_2021_NO2_1990_Y2023M04D19.TIF") as src:
    # 元数据信息
    meta = src.meta
    print("文件元数据：")
    print(f"坐标系: {src.crs}")
    print(f"分辨率: 行×列 = {src.height} × {src.width}")
    print(f"地理范围: 左={src.bounds.left}, 下={src.bounds.bottom}, 右={src.bounds.right}, 上={src.bounds.top}")
    print(f"波段数: {src.count}")
    print(f"数据类型: {src.dtypes[0]}")
    print(f"NoData值: {src.nodatavals[0]}")

文件元数据：
坐标系: EPSG:4326
分辨率: 行×列 = 16200 × 43080
地理范围: 左=-179.5512493018, 下=-60.0504162478, 右=179.4487492622, 上=74.9495832122
波段数: 1
数据类型: float32
NoData值: -999.0


In [ ]:
# prompt
# 现在我有在这个文件夹里面有很多年，一个文件夹是一年所有国家的数据，
# 请帮我批量处理好每个国家的数据，然后每个国家有不同年份的面板数据，
# 文件夹是这个“/Users/xuchengwei/Desktop/汇丰论文/空气质量/IHME_GBD_2021_AIR_POLLUTION_1990_2021_OZONE”，
# 最后给我输出一个csv包含每个国家有不同年份的面板数据


In [12]:
import os
import re
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
from rasterio.mask import mask
from rasterio.crs import CRS
from tqdm import tqdm


In [6]:
shp_path = "/Users/xuchengwei/Desktop/汇丰论文/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp"
import geopandas as gpd

# 完整文件路径
shp_path = "/Users/xuchengwei/Desktop/汇丰论文/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp"

try:
    countries = gpd.read_file(shp_path)
    print(f"✅ 成功读取数据，包含 {len(countries)} 个国家")
    
    # 查看坐标系（应为WGS84）
    print(f"\n坐标系：{countries.crs}")
    
    # 查看国家名称字段（通常为'NAME'或'ADMIN'）
    print("\n国家名称字段示例值：")
    print(countries['NAME'].head())  # 若报错，尝试使用'ADMIN'字段
    
    # 检查中国数据
    china = countries[countries['NAME'] == 'China']
    if not china.empty:
        print("\n中国数据存在，几何范围：", china.geometry.iloc[0].bounds)
    else:
        print("\n⚠️ 未找到中国数据，尝试使用其他字段匹配")

except Exception as e:
    print(f"❌ 读取失败：{e}")

✅ 成功读取数据，包含 258 个国家

坐标系：EPSG:4326

国家名称字段示例值：
0    Indonesia
1     Malaysia
2        Chile
3      Bolivia
4         Peru
Name: NAME, dtype: object

中国数据存在，几何范围： (73.60225630700006, 15.77537669500009, 134.77257938700006, 53.56944447900007)


In [8]:
import os
import re
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
from rasterio.mask import mask
from tqdm import tqdm
import matplotlib.pyplot as plt
from rasterio.crs import CRS  # 新增导入

def process_ozone_data(input_folder, countries_shapefile, output_csv, output_figures=False):
    """
    批量处理多年份臭氧数据，提取各国年均值并生成面板数据
    
    参数:
    input_folder: 包含多年份TIFF文件的文件夹路径
    countries_shapefile: 国家边界Shapefile文件路径
    output_csv: 输出面板数据CSV文件路径
    output_figures: 是否输出可视化结果
    """
    # 读取国家边界数据
    print("读取国家边界数据...")
    countries = gpd.read_file(countries_shapefile)
    
    # 检查并确保使用正确的国家名称字段
    if 'NAME' in countries.columns:
        countries = countries.rename(columns={'NAME': 'country'})
    elif 'NAME_0' in countries.columns:
        countries = countries.rename(columns={'NAME_0': 'country'})
    else:
        # 使用第一个看起来像名称的列
        name_columns = [col for col in countries.columns if 'NAME' in col or 'name' in col]
        if name_columns:
            countries = countries.rename(columns={name_columns[0]: 'country'})
        else:
            print("警告: 未找到合适的国家名称字段，将使用GID_0作为国家标识")
            countries = countries.rename(columns={'GID_0': 'country'})
    
    # 获取所有年份的TIFF文件
    tiff_files = []
    year_pattern = re.compile(r'_(\d{4})_Y\d{4}M\d{2}D\d{2}\.TIF', re.IGNORECASE)
    for file in os.listdir(input_folder):
        if file.lower().endswith('.tif'):
            match = year_pattern.search(file)
            if match:
                year = int(match.group(1))
                tiff_files.append((year, os.path.join(input_folder, file)))
    
    # 按年份排序
    tiff_files.sort(key=lambda x: x[0])
    
    if not tiff_files:
        print("未找到符合条件的TIFF文件！")
        return None
    
    print(f"找到{len(tiff_files)}个年份文件，范围: {min(t[0] for t in tiff_files)}-{max(t[0] for t in tiff_files)}")
    
    # 初始化结果列表
    panel_data = []
    
    # 批量处理每个年份
    for year, file_path in tqdm(tiff_files, desc="处理年份数据"):
        try:
            with rasterio.open(file_path) as src:
                # 获取或设置CRS（关键修改：处理无CRS的情况）
                if src.crs is None:
                    print(f"警告: {file_path}缺少CRS信息，将使用默认WGS84 (EPSG:4326)")
                    src_crs = CRS.from_epsg(4326)
                else:
                    src_crs = src.crs
                
                # 确保国家边界与栅格数据坐标系一致
                countries_proj = countries.to_crs(src_crs)
                
                # 处理多波段数据（假设为月度数据，计算年均值）
                if src.count > 1:
                    data = np.nanmean(src.read(), axis=0)  # 计算所有波段的平均值
                else:
                    data = src.read(1)
                
                # 替换无效值
                nodata = src.nodatavals[0] if src.nodatavals else np.nan
                data[data == nodata] = np.nan
                
                # 处理每个国家
                for _, country in countries_proj.iterrows():
                    country_name = country['country']
                    geometry = [country.geometry]
                    
                    try:
                        # 裁剪国家区域
                        out_image, _ = mask(src, geometry, crop=True, nodata=np.nan)
                        country_data = out_image[0] if src.count == 1 else np.nanmean(out_image, axis=0)
                        
                        # 计算国家区域内的臭氧浓度统计量
                        country_mean = np.nanmean(country_data)
                        country_min = np.nanmin(country_data)
                        country_max = np.nanmax(country_data)
                        country_std = np.nanstd(country_data)
                        
                        # 保存结果
                        panel_data.append({
                            'country': country_name,
                            'year': year,
                            'ozone_mean': country_mean,
                            'ozone_min': country_min,
                            'ozone_max': country_max,
                            'ozone_std': country_std,
                            'country_area': country.geometry.area,  # 国家面积（投影单位）
                            'data_coverage': np.sum(~np.isnan(country_data)) / country_data.size  # 数据覆盖率
                        })
                    except Exception as e:
                        print(f"处理{country_name}({year})时出错: {str(e)}")
                        continue
        except Exception as e:
            print(f"读取文件{file_path}时出错: {str(e)}")
            continue
    
    # 转换为DataFrame并保存为CSV
    if not panel_data:
        print("未处理任何数据！")
        return None
    
    panel_df = pd.DataFrame(panel_data)
    
    # 保存完整面板数据
    panel_df.to_csv(output_csv, index=False)
    print(f"面板数据已保存至: {output_csv}")
    

In [9]:
if __name__ == "__main__":
    INPUT_FOLDER = "/Users/xuchengwei/Desktop/汇丰论文/空气质量/IHME_GBD_2021_AIR_POLLUTION_1990_2021_NO2"
    COUNTRIES_SHAPEFILE = "/Users/xuchengwei/Desktop/汇丰论文/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp"
    OUTPUT_CSV = "no2_panel_data.csv"
    
    # 执行处理
    panel_data = process_ozone_data(INPUT_FOLDER, COUNTRIES_SHAPEFILE, OUTPUT_CSV, output_figures=True)
    
    # 预览结果
    if panel_data is not None:
        print("\n数据预览:")
        print(panel_data.head())
        print("\n数据统计摘要:")
        print(panel_data.describe())

读取国家边界数据...
找到18个年份文件，范围: 1990-2019


处理年份数据:   0%|          | 0/18 [00:00<?, ?it/s]/var/folders/5h/v89g_crn33g3plkrwfc455rm0000gn/T/ipykernel_75434/506756991.py:97: RuntimeWarning: Mean of empty slice
  country_mean = np.nanmean(country_data)
/var/folders/5h/v89g_crn33g3plkrwfc455rm0000gn/T/ipykernel_75434/506756991.py:98: RuntimeWarning: All-NaN slice encountered
  country_min = np.nanmin(country_data)
/var/folders/5h/v89g_crn33g3plkrwfc455rm0000gn/T/ipykernel_75434/506756991.py:99: RuntimeWarning: All-NaN slice encountered
  country_max = np.nanmax(country_data)
/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


处理Antarctica(1990)时出错: Input shapes do not overlap raster.


/var/folders/5h/v89g_crn33g3plkrwfc455rm0000gn/T/ipykernel_75434/506756991.py:97: RuntimeWarning: Mean of empty slice
  country_mean = np.nanmean(country_data)
/var/folders/5h/v89g_crn33g3plkrwfc455rm0000gn/T/ipykernel_75434/506756991.py:98: RuntimeWarning: All-NaN slice encountered
  country_min = np.nanmin(country_data)
/var/folders/5h/v89g_crn33g3plkrwfc455rm0000gn/T/ipykernel_75434/506756991.py:99: RuntimeWarning: All-NaN slice encountered
  country_max = np.nanmax(country_data)
/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/var/folders/5h/v89g_crn33g3plkrwfc455rm0000gn/T/ipykernel_75434/506756991.py:97: RuntimeWarning: Mean of empty slice
  country_mean = np.nanmean(country_data)
/var/folders/5h/v89g_crn33g3plkrwfc455rm0000gn/T/ipykernel_75434/506756991.py:98: RuntimeWarning: All-NaN slice encountered
  country_min = np.nanmin(co

处理Antarctica(1995)时出错: Input shapes do not overlap raster.


...

处理Antarctica(2018)时出错: Input shapes do not overlap raster.


/var/folders/5h/v89g_crn33g3plkrwfc455rm0000gn/T/ipykernel_75434/506756991.py:97: RuntimeWarning: Mean of empty slice
  country_mean = np.nanmean(country_data)
/var/folders/5h/v89g_crn33g3plkrwfc455rm0000gn/T/ipykernel_75434/506756991.py:98: RuntimeWarning: All-NaN slice encountered
  country_min = np.nanmin(country_data)
/var/folders/5h/v89g_crn33g3plkrwfc455rm0000gn/T/ipykernel_75434/506756991.py:99: RuntimeWarning: All-NaN slice encountered
  country_max = np.nanmax(country_data)
/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/var/folders/5h/v89g_crn33g3plkrwfc455rm0000gn/T/ipykernel_75434/506756991.py:97: RuntimeWarning: Mean of empty slice
  country_mean = np.nanmean(country_data)
/var/folders/5h/v89g_crn33g3plkrwfc455rm0000gn/T/ipykernel_75434/506756991.py:98: RuntimeWarning: All-NaN slice encountered
  country_min = np.nanmin(co

处理Antarctica(2019)时出错: Input shapes do not overlap raster.


/var/folders/5h/v89g_crn33g3plkrwfc455rm0000gn/T/ipykernel_75434/506756991.py:97: RuntimeWarning: Mean of empty slice
  country_mean = np.nanmean(country_data)
/var/folders/5h/v89g_crn33g3plkrwfc455rm0000gn/T/ipykernel_75434/506756991.py:98: RuntimeWarning: All-NaN slice encountered
  country_min = np.nanmin(country_data)
/var/folders/5h/v89g_crn33g3plkrwfc455rm0000gn/T/ipykernel_75434/506756991.py:99: RuntimeWarning: All-NaN slice encountered
  country_max = np.nanmax(country_data)
/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/var/folders/5h/v89g_crn33g3plkrwfc455rm0000gn/T/ipykernel_75434/506756991.py:97: RuntimeWarning: Mean of empty slice
  country_mean = np.nanmean(country_data)
/var/folders/5h/v89g_crn33g3plkrwfc455rm0000gn/T/ipykernel_75434/506756991.py:98: RuntimeWarning: All-NaN slice encountered
  country_min = np.nanmin(co

面板数据已保存至: no2_panel_data.csv
